In [1]:
import os

In [2]:
os.getcwd() 

'c:\\MyDLProject\\Kidney_DL_Project\\research'

In [3]:
%pwd

'c:\\MyDLProject\\Kidney_DL_Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\MyDLProject\\Kidney_DL_Project'

In [6]:
#1. Update config.yaml
#2. Update secrets.yaml [Optional]
#3. Update params.yaml
#4. Update the entity
#5. Update the configuration manager in src config
#6. Update the components
#7. Update the pipeline 
#8. Update the main.py
#9. Update the dvc.yaml
#10. app.py 


In [7]:
### step-1 Update config.yaml 

In [8]:
### step-2 Update params.yaml

In [9]:
### step-3 Update the entity 

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
### step-4 Update the configuration manager in src config

In [12]:
from src.cnnClassifier.utils.common import read_yaml,create_directories 
from src.cnnClassifier.constants import * 

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
### step-5  Update the components

In [16]:
import os 
import zipfile 
import gdown 
from src.cnnClassifier import logger 
from src.cnnClassifier.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
#### step-6 Update the pipeline 

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-05 17:21:37,007: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-05 17:21:37,009: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-05 17:21:37,011: INFO: common: created directory at: artifacts]
[2025-05-05 17:21:37,012: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-05 17:21:37,014: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=337ae27d-1230-47b0-872f-8ae40daf09dc
To: c:\MyDLProject\Kidney_DL_Project\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:14<00:00, 3.88MB/s]

[2025-05-05 17:21:57,615: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [23]:
### step-7 Main 